# grocery_sales_forecasting_NonParametric

This baseline model is non-parametric. This model is simply average of history sale (from 2013 to 2016 of the same day.)

In [1]:
# Import libraries necessary for this project
import os.path
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from IPython.display import display
import matplotlib.pyplot as plt

types = {'id': 'int32', 'item_nbr': 'int32', 'store_nbr': 'int16', 'unit_sales': 'float32', 'onpromotion': bool}

%matplotlib inline

In [2]:
#Approach1: uses the median of same item in all store with only 2017/08 data for predictation

test_data = pd.read_csv('input/test.csv', usecols=[0,2,3], dtype=types).set_index(['item_nbr','store_nbr'])
print("Favorita grocery sales forecasting testing data has {} samples with {} features each.".format(*test_data.shape))


train_data = pd.read_csv('input/train.csv', usecols=[0,2,3,4], dtype=types, 
                         converters={'unit_sales':lambda u: float(u) if float(u)>0 else 0}, 
                         skiprows=range(1,124035460)).groupby(['item_nbr','store_nbr'])['unit_sales'].median().to_frame('unit_sales')

median = test_data.join(train_data, how='left').fillna(0)

median.to_csv('median.csv', float_format='%.2f', index=None) 


Favorita grocery sales forecasting testing data has 3370464 samples with 1 features each.


/home/adou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: ParserWarning: Both a converter and dtype were specified for column unit_sales - only the converter will be used
  if __name__ == '__main__':


In [3]:
#Approach2: uses average number of the same item in the same store of previous 4 years.
#This part is computationally expensive, so the result is saved in file "test_with_hist_record.csv"
'''
years = [2013, 2014, 2015, 2016]
for year in years:
    pickle_file = 'train_' + str(year) + '_8.pkl'
    train_data = pickle.load(open( pickle_file, "rb" ))
    for item in range(test_data.shape[0]):
        date_test = test_data.at[item, 'date']
        test_store_nbr = test_data.at[item,'store_nbr']
        test_item_nbr = test_data.at[item,'item_nbr']

        date_hist = date_test.replace(year = year)
        mask = (train_data['date']==date_hist)&(train_data['store_nbr'] == test_store_nbr)&(train_data['item_nbr'] == test_item_nbr)
        result = train_data.loc[mask, 'unit_sales']
        if result.shape[0]>0:
            test_data.at[item, str(year)] = result.iloc[0]
    print(year, 'is completed!')
'''
# test_data.to_csv('test_with_hist_record.csv')


'\nyears = [2013, 2014, 2015, 2016]\nfor year in years:\n    pickle_file = \'train_\' + str(year) + \'_8.pkl\'\n    train_data = pickle.load(open( pickle_file, "rb" ))\n    for item in range(test_data.shape[0]):\n        date_test = test_data.at[item, \'date\']\n        test_store_nbr = test_data.at[item,\'store_nbr\']\n        test_item_nbr = test_data.at[item,\'item_nbr\']\n\n        date_hist = date_test.replace(year = year)\n        mask = (train_data[\'date\']==date_hist)&(train_data[\'store_nbr\'] == test_store_nbr)&(train_data[\'item_nbr\'] == test_item_nbr)\n        result = train_data.loc[mask, \'unit_sales\']\n        if result.shape[0]>0:\n            test_data.at[item, str(year)] = result.iloc[0]\n    print(year, \'is completed!\')\n'

In [7]:
# Approach2: Average and save results
test_with_hist_record = pd.read_csv('test_with_hist_record.csv', dtype=types, usecols = [1, 6,7,8,9], sep='\t')
display(test_with_hist_record.head(5))
average = test_with_hist_record[['2013', '2014', '2015', '2016']].mean(axis=1).fillna(0).to_frame('unit_sales')
average['id'] = test_with_hist_record['id']
average.set_index('id').to_csv('average.csv', float_format='%.2f') 

,id,2013,2014,2015,2016
0,125497040,3.0,NaN,NaN,NaN
1,125497041,NaN,NaN,NaN,NaN
2,125497042,NaN,NaN,NaN,NaN
3,125497043,NaN,3.0,NaN,NaN
4,125497044,2.0,NaN,1.0,NaN


In [5]:
#Load one year's train data;  
train_pkl = 'train_2017.pkl'
years = ['2013', '2014', '2015', '2016', '2017']
train_data = pickle.load(open( train_pkl, "rb" ))

#Add oil price of corresponding date; 
oil = pd.read_csv("input/oil.csv")
oil['date'] = pd.to_datetime(oil['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,oil, right_on='date',left_on='date',how='left')

#Add transaction of coresponding date and store;
transactions = pd.read_csv("input/transactions.csv")
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,transactions, left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'], how='left')
display(train_data.tail(5))
display(train_data.describe())

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dcoilwtico,transactions
23806563,125497035,2017-08-15,54,2089339,4.0,False,47.57,802
23806564,125497036,2017-08-15,54,2106464,1.0,True,47.57,802
23806565,125497037,2017-08-15,54,2110456,192.0,False,47.57,802
23806566,125497038,2017-08-15,54,2113914,198.0,True,47.57,802
23806567,125497039,2017-08-15,54,2116416,2.0,False,47.57,802


,id,store_nbr,item_nbr,unit_sales,dcoilwtico,transactions
count,2.380657e+07,2.380657e+07,2.380657e+07,2.380657e+07,1.590679e+07,2.380657e+07
mean,1.135938e+08,2.794930e+01,1.160406e+06,8.157617e+00,4.938832e+01,1.846260e+03
std,6.872364e+06,1.622054e+01,5.795121e+05,2.406447e+01,3.082288e+00,1.009838e+03
min,1.016905e+08,1.000000e+00,9.699500e+04,-1.000200e+04,4.248000e+01,9.700000e+01
25%,1.076421e+08,1.300000e+01,6.875490e+05,2.000000e+00,4.710000e+01,1.148000e+03
50%,1.135938e+08,2.800000e+01,1.176562e+06,4.000000e+00,4.922000e+01,1.513000e+03
75%,1.195454e+08,4.300000e+01,1.501544e+06,8.000000e+00,5.238000e+01,2.276000e+03
max,1.254970e+08,5.400000e+01,2.127114e+06,1.714600e+04,5.448000e+01,6.398000e+03


In [6]:
# Separate training feature and training target
target_raw = train_data['unit_sales']
features_raw = train_data.drop(['id', 'unit_sales'], axis=1)

#Drop items with missing data
features_raw.replace(["NaN", 'NaT'], np.nan, inplace = True)
features_raw.dropna(inplace=True)
# features_clean_NaN = features_raw[~features_raw.isin(['NaN', 'NaT']).any(axis=1)] #computationally expensive

#Categorical feature
features_raw['onpromotion'] = features_raw['onpromotion'].apply(lambda x: 1 if x else 0)

#Numerical feature
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['dcoilwtico', 'transactions']
features_minmax_transform = pd.DataFrame(data = features_raw)
features_minmax_transform[numerical] = scaler.fit_transform(features_minmax_transform[numerical])
display(features_minmax_transform.tail(5))
display(features_minmax_transform.describe())

,date,store_nbr,item_nbr,onpromotion,dcoilwtico,transactions
23806563,2017-08-15,54,2089339,0,0.424167,0.099941
23806564,2017-08-15,54,2106464,1,0.424167,0.099941
23806565,2017-08-15,54,2110456,0,0.424167,0.099941
23806566,2017-08-15,54,2113914,1,0.424167,0.099941
23806567,2017-08-15,54,2116416,0,0.424167,0.099941


,store_nbr,item_nbr,onpromotion,dcoilwtico,transactions
count,1.590679e+07,1.590679e+07,1.590679e+07,1.590679e+07,1.590679e+07
mean,2.790671e+01,1.158460e+06,1.252777e-01,5.756933e-01,2.852193e-01
std,1.625545e+01,5.795453e+05,3.310335e-01,2.568574e-01,1.753919e-01
min,1.000000e+00,9.699500e+04,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.300000e+01,6.837230e+05,0.000000e+00,3.850000e-01,1.606898e-01
50%,2.800000e+01,1.176558e+06,0.000000e+00,5.616667e-01,2.259455e-01
75%,4.300000e+01,1.501521e+06,0.000000e+00,8.250000e-01,3.758573e-01
max,5.400000e+01,2.127114e+06,1.000000e+00,1.000000e+00,1.000000e+00


Naive Model: Search the history record, and average it